You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}

# Importing Datasets

In [119]:
import numpy as np
import pandas as pd

In [120]:
office_addresses = pd.read_csv("datasets/office_addresses.csv")
office_addresses.head()

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [121]:
employee_addresses = pd.read_excel("datasets/employee_information.xlsx", sheet_name=0)
employee_addresses.head()

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


In [122]:
emergency_contacts = pd.read_excel("datasets/employee_information.xlsx", sheet_name="emergency_contacts", header=None, names=["employee_id", "last_name", "first_name", "emergency_contact", "emergency_contact_number", "relationship"])
emergency_contacts.head()

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [123]:
employee_roles = pd.read_json("datasets/employee_roles.json", orient="index")
employee_roles.head()

,title,monthly_salary,team
A2R5H9,CEO,$4500,Leadership
H8K0L6,CFO,$4500,Leadership
G4R7V0,Business Developer,$3000,Sales
M1Z7U9,Office Manager,$2000,People Operations


# Merging Datasets

In [124]:
employees_final = employee_addresses.merge(employee_roles, 
                                          how= "left",
                                          left_on="employee_id", 
                                          right_index=True)
employees_final.head()

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations


In [125]:
employees_final = employees_final.merge(emergency_contacts,
                                        how="left",
                                        on="employee_id")
employees_final.head()

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [126]:
employees_final = employees_final.merge(office_addresses,
                                     how="left",
                                     left_on= "employee_country",
                                     right_on= "office_country")
employees_final

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,NaN,NaN,NaN,NaN,NaN


# Cleaning Dataset

In [127]:
employees_final.set_index("employee_id", inplace=True)
employees_final

,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,,,
A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,NaN,NaN,NaN,NaN,NaN


In [128]:
office_cols = [row for row in employees_final.columns if "office" in row]
office_cols

['office',
 'office_country',
 'office_city',
 'office_street',
 'office_street_number']

In [129]:
employees_final.loc[:, office_cols] = employees_final.loc[:, office_cols].fillna("Remote")

In [130]:
employees_final

,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,,,
A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,Remote,Remote,Remote,Remote,Remote


In [131]:
employees_final.drop(["last_name", "first_name"], axis=1, inplace=True)
employees_final

,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Venus Noone,+1-202-555-0130,Wife,Remote,Remote,Remote,Remote,Remote


In [132]:
employees_final = employees_final.loc[:, ["employee_first_name",
                                        "employee_last_name",
                                        "employee_country",
                                        "employee_city",
                                        "employee_street",
                                        "employee_street_number",
                                        "emergency_contact",
                                        "emergency_contact_number",
                                        "relationship",
                                        "monthly_salary",
                                        "team",
                                        "title",
                                        "office",
                                        "office_country",
                                        "office_city",
                                        "office_street",
                                        "office_street_number"]]
employees_final

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote


In [133]:
employees_final.columns=["first_name", "last_name", "employee_country", "employee_city", "employee_street", "employee_street_number", "emergency_contact", "emergency_contact_number", "relationship", "monthly_salary", "team", "title", "office", "office_country", "office_city", "office_street", "office_street_number"]
employees_final

,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
